In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/argugpt/argugpt.csv
/kaggle/input/argugpt/machine-dev.csv
/kaggle/input/argugpt/machine-test.csv
/kaggle/input/argugpt/machine-train.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv
/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv
/kaggle/input/d/leesyyyy/models2/vocab.spm


In [2]:
import pandas as pd
import os
os.environ["KERAS_BACKEND"] = "torch"
import torch
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import keras_nlp
import keras_core as keras
import keras_core.backend as K

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using PyTorch backend.


In [3]:
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
strategy

In [4]:
train_data=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train_data['label'] = train_data.generated.copy()
# print(set(train_data['source']))
train_data

,id,prompt_id,text,generated,label
0,0059830c,0,Cars. Cars have been around since they became ...,0,0
1,005db917,0,Transportation is a large necessity in most co...,0,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0,0
3,00940276,0,How often do you ride in a car? Do you drive a...,0,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0,0
...,...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0,0


In [5]:
keras.utils.set_random_seed(42)

In [6]:
ext_df1 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')
ext_df2 = pd.read_csv('/kaggle/input/argugpt/argugpt.csv')[['id','text','model']]

ext_df2.rename(columns={'model':'source'}, inplace=True)
ext_df2['label'] = 1

ext_df = pd.concat([
    ext_df1[ext_df1.source=='persuade_corpus'].sample(10000),
    ext_df1[ext_df1.source!='persuade_corpus'],
#     ext_df2,
])
ext_df

,essay_id,text,label,source,prompt,fold
20120,83BB96F966DB,Students would benefit from being able to atte...,0,persuade_corpus,NaN,2
43529,80D6E256AA49,Landform or alien? The face on Mars looks pret...,0,persuade_corpus,NaN,2
2942,4A028F63CFCA,The human body is a complex network of muscles...,0,persuade_corpus,NaN,7
2977,5782F64504B9,Summer projects are a means by which schools a...,0,persuade_corpus,NaN,9
31921,97DD2D770B03,"Dear Mr. Senator,\n\nI have decided to express...",0,persuade_corpus,NaN,6
...,...,...,...,...,...,...
44190,falcon_180b_v1_261,I think it's a good idea for schools to have o...,1,falcon_180b_v1,Some schools have implemented policies that al...,0
44195,BDF13939FA26,Students often debate whether inactivity or s...,1,mistral7binstruct_v2,\nTask: Research the benefits of staying occup...,3
44197,1ed36fd42635_04112023,The Seagoing Cowboys program is an amazing opp...,1,darragh_claude_v6,NaN,5
44199,d8f15f7d,Advantages of Limiting Car Usage\n\nLimiting c...,1,radek_500,NaN,0


In [7]:
train = ext_df.copy().reset_index(drop=True) # pd.concat([ext_df, df], axis=0)
train.head()

,essay_id,text,label,source,prompt,fold
0,83BB96F966DB,Students would benefit from being able to atte...,0,persuade_corpus,NaN,2
1,80D6E256AA49,Landform or alien? The face on Mars looks pret...,0,persuade_corpus,NaN,2
2,4A028F63CFCA,The human body is a complex network of muscles...,0,persuade_corpus,NaN,7
3,5782F64504B9,Summer projects are a means by which schools a...,0,persuade_corpus,NaN,9
4,97DD2D770B03,"Dear Mr. Senator,\n\nI have decided to express...",0,persuade_corpus,NaN,6


In [8]:
# train=pd.concat([
#     train_data[train_data.label==0].groupby('prompt_name',group_keys=False)
#     .apply(lambda group: group.sample(frac=17497/len(train_data[train_data.label==0]), random_state=42)),
#     train_data[train_data.label==1]
# ])

In [9]:
# print(sum(train['label']==0))
# print(sum(train['label']==1))
# print(len(train))
# print('비율 맞춤')
# print(train[train_data.label == 0].groupby('prompt_name').size().sort_values(ascending=False))
# print(sum(train[train_data.label == 0].groupby('prompt_name').size()))
# print()
# # print(sum(train['prompt_name'].value_counts()))
# print('원래 데이터')
# print(train_data[train_data.label == 0].groupby('prompt_name').size().sort_values(ascending=False))
# print(sum(train_data[train_data.label == 0].groupby('prompt_name').size()))

In [10]:
# print(sum(train['label']==0))
# print(sum(train['label']==1))
# print(len(train))

In [11]:
batch_size=6
num_folds=5
epochs=3

In [12]:

# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
# train = train.reset_index(drop=True) 
# train['stratify'] = train.label.astype(str)+ train.source.astype(str)
# train["fold"]=-1
# for fold, (train_idx, val_idx) in enumerate(skf.split(train, train['stratify'])):
#     train.loc[val_idx, 'fold'] = fold
  
# train.groupby(["fold", "label","source"]).size()

In [13]:
from sklearn.model_selection import StratifiedKFold  # Import package

skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)  # Initialize K-Fold

train = train.reset_index(drop=True)  # Reset dataframe index

train['stratify'] = train.label.astype(str)+train.source.astype(str)

train["fold"] = -1  # New 'fold' column

# Assign folds using StratifiedKFold
for fold, (train_idx, val_idx) in enumerate(skf.split(train, train['stratify'])):
    train.loc[val_idx, 'fold'] = fold

# Display label distribution for each fold
train.groupby(["fold", "label", "source"]).size()

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


fold  label  source              
0     0      original_moth            484
             persuade_corpus         2000
             train_essays             275
      1      chat_gpt_moth            484
             darragh_claude_v6        200
                                     ... 
4     1      llama_70b_v1             234
             mistral7binstruct_v1     484
             mistral7binstruct_v2     484
             radek_500                100
             train_essays               1
Length: 63, dtype: int64

In [14]:
import sentencepiece as spm
from keras_nlp.models import DebertaV3Preprocessor

# SentencePiece 모델 파일 경로 설정
proto = '/kaggle/input/d/leesyyyy/models2/vocab.spm'
tokenizer = keras_nlp.models.DebertaV3Tokenizer(proto)

# 전처리 클래스 생성
preprocessor = DebertaV3Preprocessor(
    tokenizer,  # 모델 이름
    sequence_length=200    # 최대 시퀀스 길이 (짧은 경우 패딩)
)


In [15]:
inp = preprocessor(train.text.iloc[0])  # Process text for the first row

# Display the shape of each processed output
for k, v in inp.items():
    print(k, ":", v.shape)
    
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # Preprocess text
    return (text, label) if label is not None else text

token_ids : torch.Size([200])
padding_mask : torch.Size([200])


In [16]:
def build_dataset(texts, labels=None, batch_size=32,
                  cache=False, drop_remainder=True,
                  repeat=False, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, labels)  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    ds = ds.repeat() if repeat else ds  # Repeat dataset if enabled
    opt = tf.data.Options()  # Create dataset options
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=42)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds  # Return the built dataset

In [17]:
def get_datasets(fold):
    # Split the dataset into train and validation sets
    train_df = train[train.fold!=fold].sample(frac=1)
    
    # Get training data for the specified fold
    train_texts = train_df.text.tolist()
    train_labels = train_df.label.tolist()

    # Build training dataset
    train_ds = build_dataset(train_texts, train_labels,
                             batch_size=batch_size, cache=False,
                             shuffle=True, drop_remainder=True, repeat=True)

    # Get validation data for the specified fold
    valid_df = train[train.fold==fold].sample(frac=1)
    valid_texts = valid_df.text.tolist()
    valid_labels = valid_df.label.tolist()

    # Build validation dataset
    valid_ds = build_dataset(valid_texts, valid_labels,
                             batch_size=min(batch_size, len(valid_df)), cache=False,
                             shuffle=False, drop_remainder=True, repeat=False)

    return (train_ds, train_df), (valid_ds, valid_df)

In [18]:
def get_callbacks(fold):
    callbacks = []
    ckpt_cb = keras.callbacks.ModelCheckpoint(f'fold{fold}.keras',
                                              monitor='val_auc',
                                              save_best_only=True,
                                              save_weights_only=False,
                                              mode='max')  # Get Model checkpoint callback
    callbacks.append(ckpt_cb)  # Add checkpoint callback        
    return callbacks  # Return the list of callbacks

In [19]:
backbone = keras_nlp.models.DebertaV3Backbone(
    vocabulary_size=30552,
    num_layers=4,
    num_heads=4,
    hidden_dim=256,
    intermediate_dim=512,
    max_sequence_length=128,
)

In [20]:
from keras_nlp.models import DebertaV3Classifier
def build_model():
    # Create a DebertaV3Classifier model
    backbone = keras_nlp.models.DebertaV3Backbone(
    vocabulary_size=128100,
    num_layers=12,
    num_heads=12,
    hidden_dim=768,
    intermediate_dim=3072,
    max_sequence_length=512,
    )
    
    classifier = DebertaV3Classifier(
        backbone,
        preprocessor=None,
        num_classes=1  # one output per one option, for five options total 5 outputs
    )
    inputs = classifier.input
    logits = classifier(inputs)

    # Compute final output
    outputs = keras.layers.Activation("sigmoid")(logits)
    model = keras.Model(inputs, outputs)


    # Use the learning rate schedule in the optimizer

    # Compile the model with optimizer, loss, and metrics
    model.compile(
        optimizer=keras.optimizers.AdamW(5e-6),
        loss=keras.losses.BinaryCrossentropy(label_smoothing=0.02),
        metrics=[
            keras.metrics.AUC(name="auc"),
        ],
        jit_compile=True
    )

    return model

In [21]:
model = build_model()

/opt/conda/lib/python3.10/site-packages/keras_core/src/trainers/trainer.py:166: UserWarning: `jit_compile` is not yet enabled for the PyTorch backend. Proceeding with `jit_compile=False`.
  warnings.warn(


In [22]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask        │ (None, None)      │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ token_ids           │ (None, None)      │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ deberta_v3_classif… │ (None, 1)         │ 184,42… │ padding_mask[0][0],  │
│ (DebertaV3Classifi… │                   │         │ token_ids[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation          │ (None, 1)         │       0 │ deberta_v3_classifi… │
│ (Activation)        │                   │         │                      │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 184,422,913 (703.52 MB)

 Trainable params: 184,422,913 (703.52 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
for fold in range(2,3):
    (train_ds, train_df), (valid_ds, valid_df) = get_datasets(fold)
    callbacks = get_callbacks(fold)
    print('-' * 50)
    print(f'\tFold: {fold} | Model: deberta_v3_base_en\n\tBatch Size: {batch_size}')
    print(f'\tNum Train: {len(train_df)} | Num Valid: {len(valid_df)}')
    print('-' * 50)
    
    # Clear TensorFlow session and build the model within the strategy scope
    K.clear_session()
    
    model = build_model()
        
    history = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=valid_ds,
        callbacks=callbacks,
        steps_per_epoch=int(len(train_df) / batch_size),
    )
    
    best_epoch = np.argmax(model.history.history['val_auc'])
    best_auc = model.history.history['val_auc'][best_epoch]
    best_loss = model.history.history['val_loss'][best_epoch]
    
     # Print and display best results
    print(f'\n{"=" * 17} FOLD {fold} RESULTS {"=" * 17}')
    print(f'>>>> BEST Loss  : {best_loss:.3f}\n>>>> BEST AUC   : {best_auc:.3f}\n>>>> BEST Epoch : {best_epoch}')
    print('=' * 50)

--------------------------------------------------
	Fold: 2 | Model: deberta_v3_base_en
	Batch Size: 6
	Num Train: 22568 | Num Valid: 5642
--------------------------------------------------
Epoch 1/3
3761/3761 ━━━━━━━━━━━━━━━━━━━━ 1875s 498ms/step - auc: 0.9449 - loss: 0.2810 - val_auc: 0.9965 - val_loss: 0.1174
Epoch 2/3
3761/3761 ━━━━━━━━━━━━━━━━━━━━ 1878s 499ms/step - auc: 0.9951 - loss: 0.1125 - val_auc: 0.9975 - val_loss: 0.1079
Epoch 3/3
3761/3761 ━━━━━━━━━━━━━━━━━━━━ 1876s 499ms/step - auc: 0.9968 - loss: 0.0949 - val_auc: 0.9978 - val_loss: 0.1060

================= FOLD 2 RESULTS =================
>>>> BEST Loss  : 0.106
>>>> BEST AUC   : 0.998
>>>> BEST Epoch : 2


In [24]:
# from tensorflow.keras.models import load_model
# fold=2
# model_path = f'/kaggle/working/fold2.keras'

# loaded_model = load_model(model_path)

In [25]:
sub=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
test=pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [26]:
test_texts = test['text'].tolist()

# Build test dataset
test_ds = build_dataset(test_texts,
                        batch_size=batch_size, cache=False,
                        shuffle=False, drop_remainder=False, repeat=False)

# Display the shape of each processed output for the first sample
for sample in test_ds.take(1):
    for k, v in sample.items():
        print(k, ":", v.shape)

token_ids : (3, 200)
padding_mask : (3, 200)


In [27]:
predictions = model.predict(test_ds,batch_size=batch_size,verbose=1)
# predictions = model.predict(
#     valid_ds,
#     batch_size=min(CFG.batch_size * CFG.replicas * 2, len(valid_df)), # max batch size = valid size
#     verbose=1
# )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step


In [28]:
# 예측 결과 출력
submission_df = pd.DataFrame({
    'id': test['id'],
    'generated': predictions.flatten()
})
submission_df

,id,generated
0,0000aaaa,0.991587
1,1111bbbb,0.992034
2,2222cccc,0.991366


In [29]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)